<a href="https://colab.research.google.com/github/marcelotournier/llm-t5/blob/main/llm-t5-tuning-ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tunando um modelo T5 para NLP em Português

Esse notebook é um exemplo para usar o modelo T5 para uma tarefa de geração de um título para um texto.

Modifique runtime para GPU. T4 vai rodar

In [1]:
# Testando para ver se estamos usando uma GPU compatível com CUDA
# Se esse comando gerar um erro, cheque as opções no menu "Runtime" do Colab
!nvidia-smi

Sun Jul 30 12:53:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Instalando as dependências necessárias:
!pip install\
    transformers \
    accelerate \
    evaluate \
    rouge_score \
    datasets \
    --quiet

In [3]:
import pandas as pd
from datasets import Dataset

In [4]:
DATA_PATH = "https://raw.githubusercontent.com/marcelotournier/llm-t5/main/dataset_noticias_amostra_1000.csv"

In [16]:
data = Dataset.from_pandas(pd.read_csv(DATA_PATH))

In [17]:
data = data.train_test_split(test_size=0.2)

In [18]:
data["train"][0]['summary']

'Governo cria medidas que estendem adesão ao Refis e ao Funrural'

In [19]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [20]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [23]:
import evaluate

rouge = evaluate.load("rouge")

In [24]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [25]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_news_titler_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False, # Colab must be False - TPUs don't use mixed precision training
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.725742,0.159500,0.047000,0.144000,0.143900,18.965000
2,No log,2.600777,0.175700,0.054500,0.157800,0.158300,18.935000
3,No log,2.576241,0.177700,0.058800,0.160600,0.160900,18.920000
4,No log,2.566410,0.173500,0.052500,0.156400,0.156600,18.885000


TrainOutput(global_step=400, training_loss=2.8174368286132814, metrics={'train_runtime': 338.0318, 'train_samples_per_second': 9.467, 'train_steps_per_second': 1.183, 'total_flos': 865889354514432.0, 'train_loss': 2.8174368286132814, 'epoch': 4.0})

In [34]:
# Amostra para inferencia:
amostra = """Um paciente de Ribeirão Preto (SP) com câncer metastático foi o primeiro caso do Brasil a passar por um autotransplante de pulmão. O procedimento aconteceu recentemente, no Hospital São Luiz Itaim, em São Paulo (SP). O homem não teve idade ou identidade reveladas pela instituição.
Até agora, três pessoas no país passaram por esse tipo de procedimento, onde o pulmão é retirado do paciente para remoção do tumor e, posteriormente, recolocado. Mas apenas o paciente de Ribeirão enfrentava uma metástase, que é quando o câncer se espalha pelo organismo. Ele se recuperou em menos de duas semanas.
"O risco de rejeição não existe, uma vez que o órgão reimplantado é do próprio paciente", explica Marcos Samano, cirurgião torácico do Hospital São Luiz Itaim e responsável pelo procedimento.
Segundo o médico, o caso deste paciente, especificamente, impressiona. Em 2013, ele teve diagnóstico de sarcoma sinovial na perna esquerda, fez quimioterapia e radioterapia e depois cirurgia na perna.
"Alguns anos depois, apareceram lesões nos pulmões. Foram realizadas duas cirurgias e duas ablações por radiofrequência, mas uma lesão à direita cresceu e acometeu quase o pulmão direito todo", diz Samano.
O homem foi submetido ao autotransplante e se recuperou em 12 dias, fato que também surpreendeu a equipe médico responsável pelo tratamento.
Isso porque os outros dois pacientes que passaram pelo mesmo procedimento levaram 30 dias para recuperação total e nem estavam em metástase.
"""

In [37]:
# inferencia

# Nota - O Tokenizer original do T5 não sabe lidar com o "ã" :/
# por essa razão, tive que fazer uma gambiarra aqui para permitir que ao menos
# o "a" possa ser usado no lugar
prompt = f"summarize: {amostra}".replace("ã", "a")

inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(0)

outputs = model.generate(inputs, max_new_tokens=200, do_sample=False)

converted_out = tokenizer.decode(outputs[0], skip_special_tokens=True)

Checando os resultados:

Nada mal para um pequeno round de treino em Português para um modelo tão simples! :-)

Talvez possamos ter esultados melhores, usando mais dados de treino, ou um modelo maior e pré-treinado com texto em português.

In [38]:
# Imprimindo os resultados:
print("ORIGINAL:")
print(prompt)
print("TITULO:")
print(converted_out)

ORIGINAL:
summarize: Um paciente de Ribeirao Preto (SP) com câncer metastático foi o primeiro caso do Brasil a passar por um autotransplante de pulmao. O procedimento aconteceu recentemente, no Hospital Sao Luiz Itaim, em Sao Paulo (SP). O homem nao teve idade ou identidade reveladas pela instituiçao.
Até agora, três pessoas no país passaram por esse tipo de procedimento, onde o pulmao é retirado do paciente para remoçao do tumor e, posteriormente, recolocado. Mas apenas o paciente de Ribeirao enfrentava uma metástase, que é quando o câncer se espalha pelo organismo. Ele se recuperou em menos de duas semanas.
"O risco de rejeiçao nao existe, uma vez que o órgao reimplantado é do próprio paciente", explica Marcos Samano, cirurgiao torácico do Hospital Sao Luiz Itaim e responsável pelo procedimento.
Segundo o médico, o caso deste paciente, especificamente, impressiona. Em 2013, ele teve diagnóstico de sarcoma sinovial na perna esquerda, fez quimioterapia e radioterapia e depois cirurgia 